In [ ]:
# clean train set, clean test set

noise_type = ['insert0.003', 'insert0.006', 'insert0.009',
            'moved0.003', 'moved0.006', 'moved0.009',
            'replace0.003', 'replace0.006', 'replace0.009',
            'rework0.003', 'rework0.006', 'rework0.009',
            'skip0.003', 'skip0.006', 'skip0.009']

for i in range(0, len(noise_type)):
    # prefix 1~30 까지의 train set을 준비
    train_df = pd.read_csv(concat_path + "\\" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_10_concat.csv') # change (1)
    train_df = train_df.set_index('Case')
    
    X_train = train_df.drop('label', axis=1)
    X_train = X_train.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
    y_train = train_df['label']

    result_df = pd.DataFrame(columns=['Train 파일 이름', 'Test 파일 이름', 'Model', 'Accuracy Score', 'AUC Score'])

    for m in range(1, 31):
        file_pattern = os.path.join(washed_temp, '*.csv')
        file_list = glob.glob(file_pattern)
        lis = [s for s in file_list if 'BPIC15' in s and 'target_ratio_10' in s and noise_type[i] in s] # change (2)
        lis = sorted(lis, key=lambda x: int(x.split('prefix')[1].split('_')[0]))
        df = pd.read_csv(lis[m-1])

        train_file_name =  noise_type[i] + '_BPIC15_1_' + 'target_ratio_10_concat.csv' # change (3)
        test_file_name = "prefix" + str(m) + "_" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_10_concat.csv' # change (4)

        # 각 caseid 별 첫번째 event의 timestamp에 의한 train/test set 구분
        # df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df_head = df.groupby('Case').head(1).sort_values(by='Timestamp')

        total_rows = len(df_head)
        train_length = int(total_rows * 0.8)
        train_data = df_head.iloc[:train_length]
        test_data = df_head.iloc[train_length:]

        encoder_test = Aggregate(case_id_col=['Case'], cat_cols=['Activity', 'label'], boolean=False)
        encoder_test_df = encoder_test.fit_transform(df)
        try:
            encoder_test_df['label'] = encoder_test_df['label_deviant'].apply(lambda x: 0 if x > 0 else 1)
        except KeyError:
            encoder_test_df['label'] = encoder_test_df['label_regular'].apply(lambda x: 1 if x > 0 else 0)
        try:
            encoder_test_df.drop(['label_deviant', 'label_regular'], axis=1, inplace=True)
        except KeyError:
            try:
                encoder_test_df.drop(['label_deviant'], axis=1, inplace=True)
            except KeyError:
                encoder_test_df.drop(['label_regular'], axis=1, inplace=True)

        encoder_test_df = encoder_test_df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
        encoder_test_df['set'] = np.where(encoder_test_df.index.isin(train_data['Case']), 'train', 'test')

        train = encoder_test_df[encoder_test_df['set'] == 'train']
        test = encoder_test_df[encoder_test_df['set'] == 'test']

        X_test = test.drop(['label', 'set'], axis=1)
        y_test = test['label']

        # X_test의 컬럼명을 X_train의 컬럼명과 동일하게 변경, 단 X_test의 컬럼 수는 2개이고 X_train의 컬럼 수는 32개이므로 나머지 컬럼은 0으로 채움
        missing_columns = list(set(X_train.columns) - set(X_test.columns))

        # missing_columns
        for col in missing_columns:
            X_test[col] = 0

        # 컬럼을 정렬하여 순서를 맞춤
        X_test = X_test[X_train.columns]

        models = ['Random Forest', 'XGB']
        pipelines = [
            Pipeline([('model', RandomForestClassifier())]),
            Pipeline([('model', XGBClassifier())])
        ]
        param_grids = [
            {
                'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__min_samples_split': [2, 5, 10],
                'model__min_samples_leaf': [1, 2, 4],
            },
            {
                'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__learning_rate': [0.05, 0.1, 0.3],
                'model__min_child_weight': [1, 3, 5]
            },
        ]
        for model, pipeline, param_grid in zip(models, pipelines, param_grids):
            kf = KFold(n_splits=5, shuffle=True, random_state=2024)
            grid_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='accuracy')
            if len(np.unique(y_train)) == 2:
                grid_model.fit(X_train, y_train)
                best_model = grid_model.best_estimator_
                y_test_pred = best_model.predict(X_test)

            # AUC 계산을 시도하기 전에 y_test에 두 개의 클래스가 존재하는지 확인
            if len(np.unique(y_test)) == 2:
                auc = roc_auc_score(y_test, y_test_pred)
                result_df = pd.concat([result_df, pd.DataFrame({'Train 파일 이름': [train_file_name], 'Test 파일 이름': [test_file_name],
                                                                    'Model': [model], 'AUC Score': [auc]})], ignore_index=True)
                
                save_path = str(result_path + "\\" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_10_concat.xlsx') # change (5)
                result_df.to_excel(save_path, index=False)
                print(result_df)

            else:
                print(f"Skipping AUC calculation for {test_file_name} as only one class present in y_test.")

In [ ]:
# clean train set, clean test set

noise_type = ['insert0.003', 'insert0.006', 'insert0.009',
            'moved0.003', 'moved0.006', 'moved0.009',
            'replace0.003', 'replace0.006', 'replace0.009',
            'rework0.003', 'rework0.006', 'rework0.009',
            'skip0.003', 'skip0.006', 'skip0.009']

for i in range(0, len(noise_type)):
    # prefix 1~30 까지의 train set을 준비
    train_df = pd.read_csv(concat_path + "\\" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_20_concat.csv') # change (1)
    train_df = train_df.set_index('Case')
    
    X_train = train_df.drop('label', axis=1)
    X_train = X_train.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
    y_train = train_df['label']

    result_df = pd.DataFrame(columns=['Train 파일 이름', 'Test 파일 이름', 'Model', 'Accuracy Score', 'AUC Score'])

    for m in range(1, 31):
        file_pattern = os.path.join(washed_temp, '*.csv')
        file_list = glob.glob(file_pattern)
        lis = [s for s in file_list if 'BPIC15' in s and 'target_ratio_20' in s and noise_type[i] in s] # change (2)
        lis = sorted(lis, key=lambda x: int(x.split('prefix')[1].split('_')[0]))
        df = pd.read_csv(lis[m-1])

        train_file_name =  noise_type[i] + '_BPIC15_1_' + 'target_ratio_20_concat.csv' # change (3)
        test_file_name = "prefix" + str(m) + "_" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_20_concat.csv' # change (4)

        # 각 caseid 별 첫번째 event의 timestamp에 의한 train/test set 구분
        # df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df_head = df.groupby('Case').head(1).sort_values(by='Timestamp')

        total_rows = len(df_head)
        train_length = int(total_rows * 0.8)
        train_data = df_head.iloc[:train_length]
        test_data = df_head.iloc[train_length:]

        encoder_test = Aggregate(case_id_col=['Case'], cat_cols=['Activity', 'label'], boolean=False)
        encoder_test_df = encoder_test.fit_transform(df)
        try:
            encoder_test_df['label'] = encoder_test_df['label_deviant'].apply(lambda x: 0 if x > 0 else 1)
        except KeyError:
            encoder_test_df['label'] = encoder_test_df['label_regular'].apply(lambda x: 1 if x > 0 else 0)
        try:
            encoder_test_df.drop(['label_deviant', 'label_regular'], axis=1, inplace=True)
        except KeyError:
            try:
                encoder_test_df.drop(['label_deviant'], axis=1, inplace=True)
            except KeyError:
                encoder_test_df.drop(['label_regular'], axis=1, inplace=True)

        encoder_test_df = encoder_test_df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
        encoder_test_df['set'] = np.where(encoder_test_df.index.isin(train_data['Case']), 'train', 'test')

        train = encoder_test_df[encoder_test_df['set'] == 'train']
        test = encoder_test_df[encoder_test_df['set'] == 'test']

        X_test = test.drop(['label', 'set'], axis=1)
        y_test = test['label']

        # X_test의 컬럼명을 X_train의 컬럼명과 동일하게 변경, 단 X_test의 컬럼 수는 2개이고 X_train의 컬럼 수는 32개이므로 나머지 컬럼은 0으로 채움
        missing_columns = list(set(X_train.columns) - set(X_test.columns))

        # missing_columns
        for col in missing_columns:
            X_test[col] = 0

        # 컬럼을 정렬하여 순서를 맞춤
        X_test = X_test[X_train.columns]

        models = ['Random Forest', 'XGB']
        pipelines = [
            Pipeline([('model', RandomForestClassifier())]),
            Pipeline([('model', XGBClassifier())])
        ]
        param_grids = [
            {
                'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__min_samples_split': [2, 5, 10],
                'model__min_samples_leaf': [1, 2, 4],
            },
            {
                'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__learning_rate': [0.05, 0.1, 0.3],
                'model__min_child_weight': [1, 3, 5]
            },
        ]
        for model, pipeline, param_grid in zip(models, pipelines, param_grids):
            kf = KFold(n_splits=5, shuffle=True, random_state=2024)
            grid_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='accuracy')
            if len(np.unique(y_train)) == 2:
                grid_model.fit(X_train, y_train)
                best_model = grid_model.best_estimator_
                y_test_pred = best_model.predict(X_test)

            # AUC 계산을 시도하기 전에 y_test에 두 개의 클래스가 존재하는지 확인
            if len(np.unique(y_test)) == 2:
                auc = roc_auc_score(y_test, y_test_pred)
                result_df = pd.concat([result_df, pd.DataFrame({'Train 파일 이름': [train_file_name], 'Test 파일 이름': [test_file_name],
                                                                    'Model': [model], 'AUC Score': [auc]})], ignore_index=True)
                
                save_path = str(result_path + "\\" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_20_concat.xlsx') # change (5)
                result_df.to_excel(save_path, index=False)
                print(result_df)

            else:
                print(f"Skipping AUC calculation for {test_file_name} as only one class present in y_test.")

In [ ]:
# clean train set, clean test set

noise_type = ['insert0.003', 'insert0.006', 'insert0.009',
            'moved0.003', 'moved0.006', 'moved0.009',
            'replace0.003', 'replace0.006', 'replace0.009',
            'rework0.003', 'rework0.006', 'rework0.009',
            'skip0.003', 'skip0.006', 'skip0.009']

for i in range(0, len(noise_type)):
    # prefix 1~30 까지의 train set을 준비
    train_df = pd.read_csv(concat_path + "\\" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_30_concat.csv') # change (1)
    train_df = train_df.set_index('Case')
    
    X_train = train_df.drop('label', axis=1)
    X_train = X_train.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
    y_train = train_df['label']

    result_df = pd.DataFrame(columns=['Train 파일 이름', 'Test 파일 이름', 'Model', 'Accuracy Score', 'AUC Score'])

    for m in range(1, 31):
        file_pattern = os.path.join(washed_temp, '*.csv')
        file_list = glob.glob(file_pattern)
        lis = [s for s in file_list if 'BPIC15' in s and 'target_ratio_30' in s and noise_type[i] in s] # change (2)
        lis = sorted(lis, key=lambda x: int(x.split('prefix')[1].split('_')[0]))
        df = pd.read_csv(lis[m-1])

        train_file_name =  noise_type[i] + '_BPIC15_1_' + 'target_ratio_30_concat.csv' # change (3)
        test_file_name = "prefix" + str(m) + "_" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_30_concat.csv' # change (4)

        # 각 caseid 별 첫번째 event의 timestamp에 의한 train/test set 구분
        # df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df_head = df.groupby('Case').head(1).sort_values(by='Timestamp')

        total_rows = len(df_head)
        train_length = int(total_rows * 0.8)
        train_data = df_head.iloc[:train_length]
        test_data = df_head.iloc[train_length:]

        encoder_test = Aggregate(case_id_col=['Case'], cat_cols=['Activity', 'label'], boolean=False)
        encoder_test_df = encoder_test.fit_transform(df)
        try:
            encoder_test_df['label'] = encoder_test_df['label_deviant'].apply(lambda x: 0 if x > 0 else 1)
        except KeyError:
            encoder_test_df['label'] = encoder_test_df['label_regular'].apply(lambda x: 1 if x > 0 else 0)
        try:
            encoder_test_df.drop(['label_deviant', 'label_regular'], axis=1, inplace=True)
        except KeyError:
            try:
                encoder_test_df.drop(['label_deviant'], axis=1, inplace=True)
            except KeyError:
                encoder_test_df.drop(['label_regular'], axis=1, inplace=True)

        encoder_test_df = encoder_test_df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
        encoder_test_df['set'] = np.where(encoder_test_df.index.isin(train_data['Case']), 'train', 'test')

        train = encoder_test_df[encoder_test_df['set'] == 'train']
        test = encoder_test_df[encoder_test_df['set'] == 'test']

        X_test = test.drop(['label', 'set'], axis=1)
        y_test = test['label']

        # X_test의 컬럼명을 X_train의 컬럼명과 동일하게 변경, 단 X_test의 컬럼 수는 2개이고 X_train의 컬럼 수는 32개이므로 나머지 컬럼은 0으로 채움
        missing_columns = list(set(X_train.columns) - set(X_test.columns))

        # missing_columns
        for col in missing_columns:
            X_test[col] = 0

        # 컬럼을 정렬하여 순서를 맞춤
        X_test = X_test[X_train.columns]

        models = ['Random Forest', 'XGB']
        pipelines = [
            Pipeline([('model', RandomForestClassifier())]),
            Pipeline([('model', XGBClassifier())])
        ]
        param_grids = [
            {
                'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__min_samples_split': [2, 5, 10],
                'model__min_samples_leaf': [1, 2, 4],
            },
            {
                'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__learning_rate': [0.05, 0.1, 0.3],
                'model__min_child_weight': [1, 3, 5]
            },
        ]
        for model, pipeline, param_grid in zip(models, pipelines, param_grids):
            kf = KFold(n_splits=5, shuffle=True, random_state=2024)
            grid_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='accuracy')
            if len(np.unique(y_train)) == 2:
                grid_model.fit(X_train, y_train)
                best_model = grid_model.best_estimator_
                y_test_pred = best_model.predict(X_test)

            # AUC 계산을 시도하기 전에 y_test에 두 개의 클래스가 존재하는지 확인
            if len(np.unique(y_test)) == 2:
                auc = roc_auc_score(y_test, y_test_pred)
                result_df = pd.concat([result_df, pd.DataFrame({'Train 파일 이름': [train_file_name], 'Test 파일 이름': [test_file_name],
                                                                    'Model': [model], 'AUC Score': [auc]})], ignore_index=True)
                
                save_path = str(result_path + "\\" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_30_concat.xlsx') # change (5)
                result_df.to_excel(save_path, index=False)
                print(result_df)

            else:
                print(f"Skipping AUC calculation for {test_file_name} as only one class present in y_test.")

In [ ]:
# clean train set, clean test set

noise_type = ['insert0.003', 'insert0.006', 'insert0.009',
            'moved0.003', 'moved0.006', 'moved0.009',
            'replace0.003', 'replace0.006', 'replace0.009',
            'rework0.003', 'rework0.006', 'rework0.009',
            'skip0.003', 'skip0.006', 'skip0.009']

for i in range(0, len(noise_type)):
    # prefix 1~30 까지의 train set을 준비
    train_df = pd.read_csv(concat_path + "\\" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_40_concat.csv') # change (1)
    train_df = train_df.set_index('Case')
    
    X_train = train_df.drop('label', axis=1)
    X_train = X_train.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
    y_train = train_df['label']

    result_df = pd.DataFrame(columns=['Train 파일 이름', 'Test 파일 이름', 'Model', 'Accuracy Score', 'AUC Score'])

    for m in range(1, 31):
        file_pattern = os.path.join(washed_temp, '*.csv')
        file_list = glob.glob(file_pattern)
        lis = [s for s in file_list if 'BPIC15' in s and 'target_ratio_40' in s and noise_type[i] in s] # change (2)
        lis = sorted(lis, key=lambda x: int(x.split('prefix')[1].split('_')[0]))
        df = pd.read_csv(lis[m-1])

        train_file_name =  noise_type[i] + '_BPIC15_1_' + 'target_ratio_40_concat.csv' # change (3)
        test_file_name = "prefix" + str(m) + "_" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_40_concat.csv' # change (4)

        # 각 caseid 별 첫번째 event의 timestamp에 의한 train/test set 구분
        # df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df_head = df.groupby('Case').head(1).sort_values(by='Timestamp')

        total_rows = len(df_head)
        train_length = int(total_rows * 0.8)
        train_data = df_head.iloc[:train_length]
        test_data = df_head.iloc[train_length:]

        encoder_test = Aggregate(case_id_col=['Case'], cat_cols=['Activity', 'label'], boolean=False)
        encoder_test_df = encoder_test.fit_transform(df)
        try:
            encoder_test_df['label'] = encoder_test_df['label_deviant'].apply(lambda x: 0 if x > 0 else 1)
        except KeyError:
            encoder_test_df['label'] = encoder_test_df['label_regular'].apply(lambda x: 1 if x > 0 else 0)
        try:
            encoder_test_df.drop(['label_deviant', 'label_regular'], axis=1, inplace=True)
        except KeyError:
            try:
                encoder_test_df.drop(['label_deviant'], axis=1, inplace=True)
            except KeyError:
                encoder_test_df.drop(['label_regular'], axis=1, inplace=True)

        encoder_test_df = encoder_test_df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))    
        encoder_test_df['set'] = np.where(encoder_test_df.index.isin(train_data['Case']), 'train', 'test')

        train = encoder_test_df[encoder_test_df['set'] == 'train']
        test = encoder_test_df[encoder_test_df['set'] == 'test']

        X_test = test.drop(['label', 'set'], axis=1)
        y_test = test['label']

        # X_test의 컬럼명을 X_train의 컬럼명과 동일하게 변경, 단 X_test의 컬럼 수는 2개이고 X_train의 컬럼 수는 32개이므로 나머지 컬럼은 0으로 채움
        missing_columns = list(set(X_train.columns) - set(X_test.columns))

        # missing_columns
        for col in missing_columns:
            X_test[col] = 0

        # 컬럼을 정렬하여 순서를 맞춤
        X_test = X_test[X_train.columns]

        models = ['Random Forest', 'XGB']
        pipelines = [
            Pipeline([('model', RandomForestClassifier())]),
            Pipeline([('model', XGBClassifier())])
        ]
        param_grids = [
            {
                'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__min_samples_split': [2, 5, 10],
                'model__min_samples_leaf': [1, 2, 4],
            },
            {
                'model__n_estimators': [5, 10, 20],
                'model__max_depth': [2, 4, 6, None],
                'model__learning_rate': [0.05, 0.1, 0.3],
                'model__min_child_weight': [1, 3, 5]
            },
        ]
        for model, pipeline, param_grid in zip(models, pipelines, param_grids):
            kf = KFold(n_splits=5, shuffle=True, random_state=2024)
            grid_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='accuracy')
            if len(np.unique(y_train)) == 2:
                grid_model.fit(X_train, y_train)
                best_model = grid_model.best_estimator_
                y_test_pred = best_model.predict(X_test)

            # AUC 계산을 시도하기 전에 y_test에 두 개의 클래스가 존재하는지 확인
            if len(np.unique(y_test)) == 2:
                auc = roc_auc_score(y_test, y_test_pred)
                result_df = pd.concat([result_df, pd.DataFrame({'Train 파일 이름': [train_file_name], 'Test 파일 이름': [test_file_name],
                                                                    'Model': [model], 'AUC Score': [auc]})], ignore_index=True)
                
                save_path = str(result_path + "\\" + noise_type[i] + '_BPIC15_1_' + 'target_ratio_40_concat.xlsx') # change (5)
                result_df.to_excel(save_path, index=False)
                print(result_df)

            else:
                print(f"Skipping AUC calculation for {test_file_name} as only one class present in y_test.")